**Metaclasses**

![](images/metaclass.png)
The class that defines the creation of a class. Class definitions create a class name, a class dictionary, and a list of base classes. The metaclass is responsible for taking those three arguments and creating the class. They let you intercept the class statement and provide special behaviour each time a class is defined. 

    >>> class Foobar:
    ...     pass
    ...
    >>> type(Foobar)
    <class 'type'>
    >>> foo = Foobar()
    >>> type(foo)
    <class '__main__.Foobar'>

The default metaclass is `type`. We can use type directly to make a class, without any class statement:

In [11]:
# def __new__(meta, name, bases, class_dict):
#   ...
MyClass = type('Foo', (object,), {'foo': 'bar'})
a = MyClass()
a.foo

'bar'

A metaclass is defined by inheriting form type. In the default case, a metaclass receives the contents of associated class statetments in its `__new__` method. This method is responsible for creating a class instance. Here you can modify the class information before the type is actually constructed. The metaclass has access to the name of the class, parents it inherits form, and all of the class attributes defined in the body:

In [54]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        print(meta, name, bases, class_dict)
        return type.__new__(meta, name, bases, class_dict)
    
class MyClass(metaclass=Meta):
    stuff = 123
    
    def foo(self):
        pass

c = MyClass()

<class '__main__.Meta'> MyClass () {'foo': <function MyClass.foo at 0x0000016802A8E400>, '__qualname__': 'MyClass', '__module__': '__main__', 'stuff': 123}


A simple application of metaclasses is verifying that a class was defined correctly when you are building a complex class hierarchy. The `__new__` method of metaclasses is run after the class statement’s entire body has been processed.   Often a class's validation code runs in the `__init__` method; using metaclasses for validation can raise errors much earlier. 

Validation of a class's parameter can be achieved by adding functionality to the Meta.`__new__` method. In the following example, the abstract Polygon class (whose base class is object) is not validated, but all classes which inherit from it are:

In [64]:
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        # if not the abstract base class 
        if bases != (object,):
            if class_dict["sides"] < 3:
                raise ValueError("Polygons need 3+ sides")
        return type.__new__(meta, name, bases, class_dict)

class Polygon(object, metaclass=ValidatePolygon):
    sides = None
    
    @classmethod
    def interior_angles(cls):
        return (cls.sides -2) * 90
    
class Triangle(Polygon):
    sides = 3
    
Triangle.interior_angles()

90

**Method resolution order**
When you have a hierarchy of classes, how does Python decide which method to use when  `MyClass.__mro__` will gives a tuple showing the order in which methods and attributes will be looked up:

In [13]:
class Animal(object):
    pass

class Dog(Animal):
    pass

class Labrador(Dog):
    pass

Labrador.__mro__

(__main__.Labrador, __main__.Dog, __main__.Animal, object)

The MRO in action:

In [14]:
class A(object):
    def foo(self):
        print('In A')

class B(A):
    pass

class C(A):
    def foo(self):
        print('In C')
        
class D(B, C):
    pass

d = D()
d.foo()

In C


**Mutable default arguments**

In the following example, the list object has an empty list as a default value. This assignment is carried out only when the function definition is first evaluated. 

In [13]:
def func1(item, lst=[]):
    lst.append(item)
    return lst

def func2 (item, lst=None):
    if lst is None: 
        lst = []
    lst.append(item)
    return lst

This feature gives speed and memory boosts, as in most cases you will have immutable default arguments and Python can construct them just once, instead of on every function call. Another benefit is simplicity, as it is easier to understand how the expression is evaluated and thereby make debugging easier. To avoid this, the mutable objects used as defaults should be replaced by None, and then the arguments tested for None. Otherwise a new list is created each time the function is called if a second argument isn’t provided:

In [8]:
func1('a')

['a']

In [9]:
func1('b') # que?

['a', 'b']

In [11]:
func2('a')

['a']

In [13]:
func2('b') 

['b']

**Nose and py.test** 

Third-party unittest frameworks with a lighter-weight syntax for writing tests. e.g:

In [7]:
import collections
from nose.tools import assert_equal

def compress_string(string):
    if isinstance(string, str):
        lst = []
        rpts = {k:v for k,v in collections.Counter(string).items() if v > 2}
        for i in string:
            enc = i + str(rpts.get(i))
            if i in rpts.keys() and enc not in lst:
                lst.append(enc)
            elif i not in rpts.keys():
                lst.append(i)
        return ''.join(lst)
    return string

class TestCompress:
    def test_compress(self, func):
        assert_equal(func(None), None)
        assert_equal(func(''), '')
        assert_equal(func('AABBCC'), 'AABBCC')
        assert_equal(func('AAABCCDDDD'), 'A3BCCD4')
        print('Success: test_compress')

def main():
    TestCompress().test_compress(compress_string)

main()

Success: test_compress


**Object attribute lookup**

When you change a class variable from the instance, the value of the kind variable changes only for one instance. 
Instead of changing a class variable Python creates a new instance variable with the same name. Hence, the instance variables have precedence over class variables when searching for an attribute value:

    >>> car = Vehicle('Toyota', 'Corolla')
    >>> car2 = Vehicle('Honda', 'Civic')
    >>> car.kind, car2.kind
    ('car', 'car')
    >>> car.kind = 'scrap'
    >>> car.kind, car2.kind
    ('scrap', 'car')

Care needs to be taken when working with mutable class variables (e.g., list, dict). Unlike immutable types, you can change them from an instance. The rule of thumb here is to avoid class variables unless you have a reason to use them.

Instance variables are stored as a regular dictionary. When working with attributes, you just changing a dictionary.
We can access instance dictionary by calling `__dict__` dunder (magic) method. Class attribtues can also be accessed from an instance, using `__class__` dunder method (i.e., `car.__class__.__dict__`). Dictionaries of classes are a mappingproxy object.

To fully understand lookup order you need be familiar with Descriptor Protocol. But basically, the are two types of descriptors: If an object defines both `__get__()` and `__set__()`, it is considered a data descriptor. Descriptors that only define `__get__()` are called non-data descriptors (they are typically used for methods but other uses are possible). Thus, because functions only implement `__get__`, they are called non-data descriptors.

Python uses the following order for instance attribute lookups:

    Data descriptors from class dictionary and its parents
    Instance dictionary
    Non-data descriptors from class dictionary and its parents

Keep in mind, that no matter how many levels of inheritance you have there is always one instance dictionary which stores all instance variables.

Attribute lookup on the class of an instance is not an actually attribute access, it is a bit different and is to do with the MRO. The MRO is an attribute defined on the metaclass, it doesn't show up on the instance. Lookups on the class of an instance:

    0. __getattribute__ on class
    1. data descriptor on class
    2. __dict__
    3. non-data descriptor on class
    4. simple value from the class
    5. __getattr__ on class
    6. raise AttributeError

Creating a data descriptor through defining \__get\__ and \__set\__ allows you to control the access to the attribute.  If you do not define \__set\__ (so not a data descriptor) you are free to overide the attribute in the instance so you can put the attribute in \__dict\__ (of the instance) and you will get it back from the dict before the descriptor is checked.

In [14]:
class D:
    foo = 3
    
class E(D):
    def __init__(self):
        self.foo = 5
        
e = E()
type(e).foo

3

**Polymorphism**

Refers to the ability of an object to provide different behaviors (use different implementations) depending on its own nature. Specifically, depending on its position in the class hierarchy. Polymorphism is declaring a uniform interface that isn't type aware, leaving implementation details to concrete types that implement the interface. It allows the expression of some sort of contract, with potentially many types implementing that contract (whether through class inheritance or not) in different ways, each according to their own purpose. Code using that contract should not(*) have to care about which implementation is involved, only that the contract will be obeyed.

Polymorphism can be achieved by *method overriding*,  when a method defined in a superclass or interface is re-defined by one of its subclasses, thus modifying/replacing the behavior the superclass provides. Notice the signature of the method remains the same when overriding. *Method overloading* is unrelated to polymorphism. It refers to defining different forms of a method. In this case the signature of the method is changed. 

**Property decorator and the uniform access principle** 

Python code strives to adhere to the Uniform Access Principle; the are no truly 'protected' or 'private' attributes. Getters and setters are used in many object oriented programming languages to ensure the principle of data encapsulation (the bundling of data with the methods that operate on these data). The accepted approach in Python is to xxpose your instance variables directly, e.g. foo.x = 0, not foo.set_x(0), which preserves the access semantics. The main advantage to this approach is that the caller gets to do this: foo.x += 1 instead of the less-readable: foo.set_x(foo.get_x() + 1)

Instance variables starting with a single underscore are conventionally private; not to be messed with directly. and they shouldn't mess with it directly. With double underscore, Python mangles the name but the variable is still accessible from outside.

If you need to wrap the access variables assigned by methods use `@property`. Getter, setter and deleter methods enable you to set an attribute using a function. Getting access is the same and the setter method allows you to have functionality of normal class attribute assignment. You can start with the simplest implementation imaginable, and you are free to later migrate to a version which preserves the access semantics and so avoids having to change the interface.

Specifying a `setter` on a property also lets you perform type checking and validation on values passed to the class. See the 'Validation of class attributes' section below for the difference between property decorators and descriptors for attribute validation.

In [15]:
class A(object):
    _x = 0
    '''A._x is an attribute'''

    @property
    def x(self):
        '''
        A.x is a property
        This is the getter method
        '''
        return self._x

    @x.setter
    def x(self, value):
        """
        This is the setter method
        where I can check it's not assigned a value < 0
        """
        if value < 0:
            raise ValueError("Must be >= 0")
        self._x = value

a = A()
a._x = -1 
a.x = 2
# raises ValueError:
# a.x = -1 

In [32]:
class Resistor(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0   

        
class VoltageResistor(Resistor):
    """ Migrate to property decorator to allow current to be varied by assigning the voltage property """
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
        
    @property 
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms
        
        
class FixedResistor(Resistor):
    """ Use @property to make attribtues from parent classes immutable """
    def __init__(self, ohms):
        super().__init__(ohms)
        self._ohms = ohms
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, 'ohms'):
            raise AttributeError("Can't set attribute")
        self._ohms = ohms
        
r1 = Resistor(500)
r2 = VoltageResistor(500)
r3 = FixedResistor(500)

# Immutable, would raise AttributeError:
# r3.ohms = 5
r2.voltage = 12
r2.current

0.024

**Recursion**

A function is recursive if it calls itself and has a termination condition. Why a termination condition? To stop the function from calling itself ad infinity. An example of recursion in English: “A human is someone whose mother is human”. Also, a tree diagram where each branch is like a new tree.

The two key elements of a recursive algorithm are:

The termination condition: n == 0
The reduction step where the function calls itself with a smaller number each time: factorial(n - 1)

In [2]:
import logging

def factorial(n):
    if n == 0:
        return 1
    else:
        logging.warning(n)
        return n * factorial(n - 1)

factorial(3)

6

**Scope and closures**

One of the most fundamental paradigms of nearly all programming languages is the ability to store values in variables, and later retrieve or modify those values. In fact, the ability to store values and pull values out of variables is what gives a program state. 

But the inclusion of variables into our program begets the most interesting questions we will now address: where do those variables live? In other words, where are they stored? And, most importantly, how does our program find them when it needs them? These questions speak to the need for a well-defined set of rules for storing variables in some location, and for finding those variables at a later time. We'll call that set of rules: Scope.

Python has *lexical scoping* whereby a variable may only be called or referenced from within the block of code in which it is defined. Whilst functions might be the basic unit of scope declaration, there are other blocks of code that define scopes, e.g. control flow and loop blocks. The same identifier name can be specified at multiple layers of nested scope, which is called “shadowing”. Regardless of shadowing, scope look-up always starts at the innermost scope being executed at the time, and works its way outward/upward until the first match, and stops.

A closure function is any function that uses a variable that is defined in an environment (or scope) that is external to that function, and is accessible within the function when invoked from a scope in which that free variable is not defined. Closures in computer programming are generally useful as callback functions. Python provides support for closures via lexical scoping. Use-cases include lazy evaluation. 

![](images/closures.png)

In the first example, `mult6` and `mult7` are function closures. They are functions because `make_multiplier` returns the function `multiply`. It is a closure because the returned function refers to the free variable factor. The factor variable is local in the scope of `make_multiplier`, but not in the scope of `multiply`. Since this variable is referenced by the returned function, Python knows it needs to store that variable along with the returned function, and not garbage-collect it once `make_multiplier` completes.

Even if a variable named factor was defined in the calling scope, it has nothing to do with the factor variable in the closure.  In the second example, `main` is a function with a local variable named `factor`. The local `factor` and the closure `factor` have nothing to do with each other. For each instance of the `multiply` closure, factor is the one from the closure, so the results of invoking it are unchanged. In main, the `factor` variable is assigned *10* and not changed, so it is still *10* when printed at the end.

In its nature, Python doesn’t have variable declaration semantics. A variable is “declared” when it’s name is bound to an object. Contrast with JavaScript e.g. `function myFunc(){var a; a = 2; alert(a)}`. One caveat is that it is an error to delete a name of a variable that is referenced in an enclosing scope.  If Python needs to be able to access factor when mult6 is invoked, it cannot have it deleted!

**Strong-typing** You cannot coerce objects into a different type by any kind of inferring, as in weakly-typed languages. As so, the object type is more explicit.



**`super()`**

For simple inheritance, allows parent classes to be accessed through child classes without having to hardcode the name of the parent class.

`super()` calls only one class `__init__`:

In [36]:
class A:
    def __init__(self):
        print("A")

class B:
    def __init__(self):
        print("B")

class C(B, A):
    def __init__(self):
        super().__init__() 

c=C() # n.b. is defined as class C(A, B) it would reach class A first and print "A"

B


Defining `super().__init__()` in all the classes will resolve the problem. In the following example, the `__init__` method of A is called first and then the `super()` function in A is called so it goes to class B and executes both print functions before printing "A":

In [38]:
class A:
    def __init__(self):
        print("reached A")
        super().__init__()
        print("A")

class B:
    def __init__(self):
        print("reached B")
        super().__init__()
        print("B")

class C(A,B):
    def __init__(self):
        super().__init__() 

c=C()

reached A
reached B
B
A


**Validation of class attributes** 

This can be done using property decorators or with descriptors

In [43]:
class Rectangle:
    def __init__(self, length, width):
        self.length = length #if self._lenght is used, then it will not validate through setter. 
        self.width = width

    @property
    def area(self):
        return self.length * self.width

    def perimeter(self):
        return 2 * (self.length + self.width)

    @property
    def length(self):
        return self._length

    @length.setter
    def length(self, value):
        if not isinstance(value, int): #validating length
            raise TypeError("Only integers are allowed")
        self._length = value

r = Rectangle(3,2)
r.length = 4
r.area

8

We may also want to provide validation for multiple attributes (e.g. width as well as length). Descriptors would help  useful in this case:

In [51]:
class Integer: 
    def __init__(self, parameter):
        self.parameter = parameter

    def __get__(self, instance, owner):
        if instance is None: 
            return self
        else: 
            return instance.__dict__[self.parameter]

    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError("Interger value is expected")
        instance.__dict__[self.parameter] = value

class Rect: 
    length = Integer('length')
    width =  Integer('width')

    def __init__(self, length, width):
        self.length = length
        self.width = width

r = Rect(2, 1)
r.length

2